In [114]:
import numpy as np
import pandas as pd
import numpy_financial as npf
from datetime import date

In [116]:
Interest_Rate = 0.04
Years = 30
Payments_Year = 12 # monthly payments
Principal = 200000
start_date = date(2016, 1, 1)

In [117]:
# Payment against loan principal plus interest

# As the payment is monthly the interest rate is divided and the years is multiply. The payment is done at the end of the month.

# when="end" is default
monthly_payment = npf.pmt(Interest_Rate/Payments_Year, Years*Payments_Year, Principal, when="end")
print(monthly_payment)

-954.8305909309076


In [118]:
total_payment = monthly_payment * 12 * Years
total_payment

-343739.0127351268

In [119]:
# Interest portion of a payment
per = 1 # first installment

interest_payment = npf.ipmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)

interest_payment

array(-666.66666667)

In [120]:
# Principal portion of a payment
principal_payment = npf.ppmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)

principal_payment

-288.1639242642409

In [121]:
monthly_payment == interest_payment + principal_payment

True

In [159]:
data = {
  "month": range(1, Years * 12 + 1),
  "int_pay": [],
  "pri_pay": [],
  # "total_installment": []
}

for month in data["month"]:
  data["int_pay"].append(npf.ipmt(Interest_Rate/Payments_Year, month, Years*Payments_Year, -Principal))

  data["pri_pay"].append(npf.ppmt(Interest_Rate/Payments_Year, month, Years*Payments_Year, -Principal))

  # data["total_installment"].append(-monthly_payment)

len(data["pri_pay"])

360

In [160]:
emi = pd.DataFrame(data)
emi.set_index("month", inplace=True)

emi["total_monthly"] = emi["int_pay"] + emi["pri_pay"]

print(emi)

                  int_pay     pri_pay total_monthly
month                                              
1       666.6666666666667  288.163924    954.830591
2       665.7061202524525  289.124471    954.830591
3       664.7423720168576  290.088219    954.830591
4       663.7754112871442  291.055180    954.830591
5       662.8052273549982  292.025364    954.830591
...                   ...         ...           ...
356    15.755934291332574  939.074657    954.830591
357    12.625685435867249  942.204905    954.830591
358     9.485002417550422  945.345589    954.830591
359     6.333850455839032  948.496740    954.830591
360    3.1721946542554846  951.658396    954.830591

[360 rows x 3 columns]


## Better to create a function

In [170]:
def amort_table(rate, years, payment_year, principal, start_date):
  """
  Amort_table uses numpy_financial under the hood to provide am amortization table for your emis.

  Args:
    rate: in percentage, Eg -> `0.05 == 5%`
    years: no. of years
    payment_year: monthly interest = `12` or semi-annually = `2`
    principal: total amount to be paid
    start_date: `date(year, month, day)`

  Returns:
    DataFrame of EMI Payments
  """
  # freq="MS" Month Start
  date_range = pd.date_range(start_date, periods=years*payment_year, freq="MS")

  # print(date_range)

  df_emi = pd.DataFrame(index=date_range)
  df_emi.reset_index(inplace=True)
  df_emi.index += 1
  df_emi.index.name = "Period"

  df_emi["Interest_Pay"] = npf.ipmt(rate/payment_year, df_emi.index, years*payment_year, principal)

  df_emi["Principal_Pay"] = npf.ppmt(rate/payment_year, df_emi.index, years*payment_year, principal)

  # df_emi["Payment"] = npf.pmt(rate/payment_year, years*payment_year, principal)

  df_emi["Total_Pay"] = df_emi["Interest_Pay"] + df_emi["Principal_Pay"]

  return df_emi.loc[:, "Interest_Pay":]


In [171]:
que1 = amort_table(0.04, 30, 12, -200000, date(2016, 1, 1))
print(que1)

        Interest_Pay  Principal_Pay   Total_Pay
Period                                         
1         666.666667     288.163924  954.830591
2         665.706120     289.124471  954.830591
3         664.742372     290.088219  954.830591
4         663.775411     291.055180  954.830591
5         662.805227     292.025364  954.830591
...              ...            ...         ...
356        15.755934     939.074657  954.830591
357        12.625685     942.204905  954.830591
358         9.485002     945.345589  954.830591
359         6.333850     948.496740  954.830591
360         3.172195     951.658396  954.830591

[360 rows x 3 columns]
